<a href="https://colab.research.google.com/github/dgromann/ProgrammingForTranslators/blob/master/4_Beispiel_Termextraktion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programmieren für ÜbersetzerInnen - Beispiel Termextraktion




Als erstes Beispiel nach der Einführung schreiben wir ein simples Programm zur Termextraktion unter Verwendung des statistischen Maßes TF-IDF. TF-IDF steht für Termhäufigkeit (TF) und Inverse Dokumentenhäufigkeit (IDF) und ist ein Maß zur Berechnung der Gewichtung eines Wortes/einer Phrase in einem Dokument -  damit können fachsprachliche Benennungen extrahiert werden.

Unser Beispielkorpus besteht aus den folgenden drei Sätzen: 


```
document1 = "Sendung von Gold in einem Container." 
document2= "Lieferung von Silber in einem silbernen LKW angekommen." 
document3 = "Sendung von Gold in einem LKW angekommen."
```
Anstatt diese Sätze direkt im Notebook zu schreiben, laden wir drei Dokumente in unser Programm:


In [0]:
# Datein in Google Colab laden
from google.colab import files
files.upload()

document1 = open("document1.txt")
document2= open("document2.txt") 
document3 = open("document3.txt")
corpus = [document1.readlines(), document2.readlines(), document3.readlines()]

#Wie Sie sehen wird für jedes Dokument eine Liste erstellt, die jede Zeile (in diesem Fall nur eine), des Dokuments enthält.
print(corpus)

Um spaCy auch in diesem Notebook verwenden zu können, müssen wir spaCy und auch das deutsche Sprachpaket wieder laden:

In [0]:
# spaCy in Colab/Binder installieren - für Anaconda siehe spacy Installatiosnanweisungen
!pip install -U spacy
!python -m spacy download en
!python -m spacy download de

import spacy
nlp = spacy.load('de')

Wir verwenden hier die Vorverarbeitungsmethode aus dem letzten Beispiel. Da diese in einem anderen Notebook steht, müssen wir Sie hier noch einmal wiedergeben: 

In [0]:
def preprocessing(sentence):
  sentence = sentence.lower() 
  # Wir fügen auch gleich ein paar weitere POS-Tags für Konjunktionen, etc. hinzu 
  pos_to_be_removed =['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE']
  text_out = []
  # Tokenisiert und weitere Vorverarbeitung
  doc= nlp(sentence)
  for token in doc:
    # POS-Tags überprüft und nur jene die nicht in "pos_to_be_removed" zu finden sind berücksichtigen
    if token.pos_ not in pos_to_be_removed :
      #Lemmatisierung
      lemma = token.lemma_
      text_out.append(lemma)
    return text_out

**Aufgabe** \\
Rufen Sie die oben definierte Methode `preprocessing` für jeden Satz des Korpus `corpus` auf und speichern Sie das Ergebnis in eine neue Liste `preprocessed`:

In [12]:
# neue Liste preprocessed 
preprocessed = []

for sentence in corpus: 
  print(sentence)
  preprocessing(sentence)
# jedes Dokument (Satz) in der Variable corpus durch die Metehodee preprocessing bearbeiten


['Sendung von Gold in einem Container.\n']


AttributeError: ignored

# TF-IDF
Anstelle der manuellen Berechnung der TF-IDF Werte, verwenden wir eine bereitgestellte Library für maschinelles Lernen names `sklearn`. 



In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)
word_index = vectorizer.vocabulary_

print("Wortindex: ", word_index)
print("TF-IDF Matrix: ")
print("(Dokumentennummer, Termindex) TF-IDF Weight")
print(tfidf_matrix)

Der nachstehende Code extrahiert die `n` wichtigsten Terme aus unserem Mini-Korpus. 



**Frage** \\
Würden Sie sagen, dass diese Auswahl für diesen Mini-Korpus Sinn macht? Stellen diese Worte tatsächlich wichtige semantische Bestandteile unseres Korpus dar?

In [0]:
import numpy as np

feature_array = np.array(vectorizer.get_feature_names())
tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]
print(top_n)